In [1]:
%run -i "ocr_query_merger.ipynb"

In [2]:
import import_ipynb
from difflib import SequenceMatcher
from faker import Faker
import random
import faker_commerce
from PIL import Image, ImageFont, ImageDraw, ImageFilter
import import_ipynb
import time
fake = Faker()
fake.add_provider(faker_commerce.Provider)

In [3]:
def table_data_generator(headers,max_entries,max_code_len,units,code_type):
    table = []
    number_of_rows = random.randint(1,max_entries)
    for i in range(number_of_rows):
        row_entry = []
        for j in headers:
            if j == 'index':
                row_entry.append(i+1)
            elif j == 'code':
                if code_type == 'None':
                    row_entry.append(fake.password(length=max_code_len,special_chars=False,upper_case=True))
                elif code_type == 'int':
                    row_entry.append(fake.random_int(min=10**max_code_len, max=10**(max_code_len+1)-1))
                elif code_type == 'special56':
                    row_entry.append(fake.pystr_format('?-??-###-???','ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
            elif j == 'desc':
                row_entry.append(fake.ecommerce_name()) 
            elif j == 'unit':
                if units:
                    row_entry.append(str(fake.random_int(min=1, max=100))+" PCS")
                else:
                    row_entry.append(fake.random_int(min=1, max=100))
            elif j == 'float':
                row_entry.append(fake.pyfloat(right_digits = 2,positive=True, min_value=0.1, max_value=100))
        table.append(row_entry)
    return table

In [4]:
def table_image_generator(data):
    table = table_data_generator(headers=data['headers'],max_entries=data['max_entries'],max_code_len=data['max_code_len'],units=data['units'],code_type=data['code_type'])
    T_table = [[j[i] for j in table] for i in range(len(table[0]))]
    font = ImageFont.truetype(font = 'arial',size = data['textsize'])
    image = Image.open(data['template'])
    img = ImageDraw.Draw(image)
    line_spacing = 10
    
    for i in range(len(T_table)):
        position = data['position'][i]
        for j in T_table[i]:
            string = str(j)
            if len(string)>data['max_text_len']:
                string = string[:20:]
            img.text(position,string,font=font,fill=(0, 0, 0))
            position = (position[0],position[1]+data['spacing'])
    
#     rotation_angle = random.uniform(-2,2)
    rotation_angle = 0
    rotation_center = [random.uniform(image.size[0],image.size[1]) for _ in range(2)]
#     edge_enhance_level = random.randint(0,2)
    edge_enhance_level = 0
    for i in range(edge_enhance_level):
        image = image.filter(ImageFilter.EDGE_ENHANCE)
    image = image.rotate(rotation_angle,center = rotation_center,expand=True)
#     image.show()
    return {'image':image,'rot_angle':rotation_angle,'enhance_level':edge_enhance_level,'test_data':table}

In [5]:
def run_test(iterations):
    R_4_17_headers = ['index','desc','unit','float','float']
    R_4_17_position = [(15,460),(120,460),(754,460),(930,460),(1222,460)]
    R_4_17_template = '../templates/R-4-17.jpg'
    R_4_17 = {'headers':R_4_17_headers,'position':R_4_17_position,'template':R_4_17_template,'textsize':25,'spacing':63,'max_text_len':13,'code_type':'None','max_code_len':8,'max_entries':12,'units':True}

    R_4_43_headers = ['code','desc','unit','float','float','float']
    R_4_43_position = [(137,915),(268,915),(1074,920),(1225,915),(1432,915),(1565,915)]
    R_4_43_template = '../templates/R-4-43.jpg'
    R_4_43 = {'headers':R_4_43_headers,'position':R_4_43_position,'template':R_4_43_template,'textsize':30,'spacing':35,'max_code_len':5,'code_type':'None','max_text_len':13,'max_entries':15,'units':True}

    R_4_46_headers = ['index','code','desc','unit','float','float']
    R_4_46_position = [(120,690),(190,690),(417,690),(1082,690),(1175,690),(1340,690)]
    R_4_46_template = '../templates/R-4-46.jpg'
    R_4_46 = {'headers':R_4_46_headers,'position':R_4_46_position,'template':R_4_46_template,'textsize':24,'spacing':40,'max_code_len':13,'code_type':'int','max_text_len':13,'max_entries':15,'units':False}

    R_4_47_headers = ['index','desc','unit','float','float']
    R_4_47_position = [(198,840),(269,840),(1050,840),(1192,840),(1500,840)]
    R_4_47_template = '../templates/R-4-47.jpg'
    R_4_47 = {'headers':R_4_47_headers,'position':R_4_47_position,'template':R_4_47_template,'textsize':24,'spacing':61,'max_code_len':13,'code_type':'int','max_text_len':13,'max_entries':12,'units':True}

    R_4_56_headers = ['index','code','desc','unit','float','float']
    R_4_56_position = [(53,486),(97,486),(348,486),(1073,486),(1293,486),(1550,486)]
    R_4_56_template = '../templates/R-4-56.jpg'
    R_4_56 = {'headers':R_4_56_headers,'position':R_4_56_position,'template':R_4_56_template,'textsize':26,'spacing':40,'max_code_len':13,'code_type':'special56','max_text_len':13,'max_entries':20,'units':True}
    
    template_options = [R_4_17,R_4_43,R_4_46,R_4_47,R_4_56]
    result_str = []
    
    for i in range(iterations):
        start_time = time.time()
        test = table_image_generator(template_options[random.randint(0,len(template_options)-1)])
        results = run_ocr_merger(test['image'],type_of_document='invoice')
        time_taken = time.time() - start_time
        print("Test ",i+1," Parameters")
        print("Rotation Angle: ",test['rot_angle'],"\tEnhance Level: ", test['enhance_level'])
        print("Results")
        if results:
            table_found = 1
            count = 0
            similarity = 0
            history=[]
            for j in np.array(test['test_data']).flatten().tolist():
                for k in np.array(results['table_data']['row_data']).flatten().tolist()[:len(np.array(test['test_data']).flatten().tolist()):]:
                    if k!=None and k not in history and (str(j).casefold() in str(k).casefold() or str(k).casefold() in str(j).casefold() or SequenceMatcher(None, str(j).casefold(), str(k).casefold()).ratio()>0.6):
                        similarity+=1
                        history.append(k)
                        break
            similarity = 100*similarity/len(np.array(test['test_data']).flatten().tolist())
            
            print("% Similarity: ",similarity,"%",end='\t')
            count = 0
            accuracy = 0
            for row in range(len(test['test_data'])):
                for col in range(len(test['test_data'][0])):
                    if len(results['table_data']['row_data'])-1>row and len(results['table_data']['row_data'][row])-1>col: 
                        count+=1
                        if test['test_data'][row][col] == results['table_data']['row_data'][row][col]:
                            accuracy+=1
            if count!=0:
                accuracy = 100*accuracy/count
            else:
                accuracy = 0
            print("% Accuracy: ",accuracy,"%")
        else:
            table_found = 0
            similarity = 0
            accuracy = 0
            print("Error. Table not found.")
        print("Time Taken: %s seconds" % (time.time() - start_time))
        print("-----------------------------------------------------")
        print([print(k) for k in results['table_data']['row_data']])
        print()
        result_str.append(f"{i},{test['rot_angle']},{test['enhance_level']},{table_found},{similarity},{accuracy},{time_taken},{results}")
        if len(result_str)>2 or i == iterations-1:
            try:
                open("test_log.csv", "r")
            except:
                f = open("test_log.csv", "a")
                f.write("Test Number,Rotation Angle,Enhance Level, Table Found,Similarity,Accuracy,Time Taken,Results\n")
            else:
                f = open("test_log.csv", "a")
            for j in result_str:
                f.write(j)
                f.write('\n')
            f.close()
            result_str = []
        print("-----------------------------------------------------")
        

In [ ]:
errors = 0
while(1):
    try:
        run_test(1)
    except:
        errors+=1
        continue

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  87.03703703703704 %	% Accuracy:  45.0 %
Time Taken: 24.954381465911865 seconds
-----------------------------------------------------
['3ADTg i6TvN', 'Licensed Fish', '68 PCS', '2.42 47.95', '53.9 91.65', '23.3 76.8']
[None, 'Cotton Bacon Steel Keyboard', '40 PCS', None, None, None]
['6Dn87', None, '83 PCS', '33.61', '10.9', '67.88']
['3vVIy', 'Plastic Pizza', '57 PCS', '59.81', '50.42', '18.38']
['9MCDo', 'Ergonomic Wooden Bik', '12 PCS', '4.73', '19.26', '95.36']
['6Phee', 'Ergonomic Car', '49 PCS', '93.1', '96.88', '1.57']
['0HHUw', 'Used Concrete Hat', '58 PCS', '4.36', '30.59', '66.9']
['3OSij', 'Keyboard Gorgeous Hat', '97 PCS', '93.57', '94.37 62.5', '74.51']
['4DR3e', None, '45 PCS', '48.17', None, '99.21']
[None, None, None, None, None, None, None, None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  72.5

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  62.857142857142854 %	% Accuracy:  0.0 %
Time Taken: 26.283946990966797 seconds
-----------------------------------------------------
[None, '71 PCS', '70.81', None, '17.67']
['Rubber Pizza', '9 PCS', '42.31', None, '78.99']
['Rustic Concrete Ball', '63 PCS', '95.15', None, '21.8']
[None, '84 PCS', '24.22', None, '46.89']
['Intelligent Rubber K', '22 PCS', '6.85', None, '27.11']
['Unbranded Wooden Fis', '33 PCS', '40.11', None, '97.74']
[None, None, None, None, None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  100.0 %	% Accuracy:  60.0 %
Time Taken: 20.710322856903076 seconds
-----------------------------------------------------
['8Lo97', 'Keyboard', '8 PCS', '2.68', '66.43', '30.21']
['41MEi', 'Sleek Soft Chicken', '24 PCS', '13.77', '60.84', '93.4']
[None, None]

----------------------------------------------

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  90.0 %	% Accuracy:  12.0 %
Time Taken: 15.789146184921265 seconds
-----------------------------------------------------
[None, '56200238728323', 'Fantastic Chicken', '30', None, '79.18']
['2', '33889309423443', 'Practical Plastic Sa', '52', '31.35', '26.66']
['3', '30294364566767', 'Gently Used Fresh Ca', '86', '18.38', '71.47']
['4', '70881469517213', 'Car', '86', '90.57', '23.53']
['5', '70001841578320', 'Fresh Chicken', '58', '44.14', '10.27']
[None, None, None, None, 'Amount', '112.50']
[None, None, None, 'GST @ 7 %', None, '7.88']
[None, 'One hundred twenty dollars and thirty-elght cents only', None, 'Net Amount', None, None]
[None, None, None, None, None, None, None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  83.33333333333333 %	% Accuracy:  0.0 %
Time Taken: 25.724271059036255 seconds
-----------------

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  70.0 %	% Accuracy:  0.0 %
Time Taken: 25.41348886489868 seconds
-----------------------------------------------------
[None, '64 PCS', '10.3', None, '9.42']
[None, '92 PCS', '85.26', None, '99.33']
['Practical Bike', '4 PCS', '68.1', None, '22.44']
['Practical Steel Comp', '4 PCS', '51.16', None, '6.1']
['Used Soft Pizza', '96 PCS', '80.9', None, '22.56']
['Incredible Granite C', '75 PCS', '38.78', None, '60.22']
['Awesome Fish', '24 PCS', '93.49', None, '25.31']
[None, '37 PCS', '88.66', None, '9.4']
[None, None, 'Goods Totat', None, '140.28']
['Details for payment via hank transer-', None, 'Less Discount Gocs Amount', '%', '140.28']
['Bank NameUnited Overseas Bank LimitedA/C Name:Naspac Marketing Pe Ltd', None, '7%GST', None, None]
['A/c No.3013534215for SGD)/301 901 0137(for USDSwt Cod-UOVBSGSCXXX', None, 'atalincl GST', None, None]
['neweccn', None, None, None, 'Enof Dixament']
['All cheques should be'

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  95.83333333333333 %	% Accuracy:  46.666666666666664 %
Time Taken: 22.060895919799805 seconds
-----------------------------------------------------
['0SyyK', 'New Tuna', '68 PCS', '85.83', '55.24', '70.27']
['4Rct2', 'For repair Wooden Ch', '97 PCS', '36.66', '23.53', '45.75']
['1FIQt 0RMap', 'Refined Chair', '25 PCS', '65.74', '97.63', '26.21']
[None, 'Generic Chips', '85 PCS', '92.31', '65.84', '32.27']
[None, None, None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  66.66666666666667 %	% Accuracy:  0.0 %
Time Taken: 27.2839252948761 seconds
-----------------------------------------------------
[None, '30 PCS', '39.39', None, '11.51']
[None, '100 PCS', '85.96', None, '72.31']
['Used Chips', '9 PCS', '24.1', None, '85.13']
['Tasty Fresh Computer', '56 PCS', '21.5', None, '28.61']
[None, '38 PCS', '39.79', None, 

...

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  96.42857142857143 %	% Accuracy:  46.15384615384615 %
Time Taken: 32.718894720077515 seconds
-----------------------------------------------------
['7Qs30', 'Frozen Shirt', '56 PCS', '36.11', '54.73', '65.77']
['3yHuS', 'Incredible Plastic S', '43 PCS', '63.54', '99.85', '2.79']
['4LkIG 5LAch', 'Towels', '57 PCS', '36.21', '65.19', '31.88']
[None, 'Gorgeous Towels Keyboard', '37 PCS', '26.93', '57.77', '45.45']
['g1Y5z', None, '5 PCS', '30.98', '92.5', '24.26']
['A7LoO', 'Cotton Pizza', '74 PCS', '52.55', '90.49', '8.35']
['3VosD', 'Soap', '85 PCS', '34.1', '22.33', '93.54']
['3EgEW', 'Small Sausages Shirt', '42 PCS', '81.46', '77.77', '71.39']
['n4L8r', 'Chicken', '80 PCS', '61.88', '61.86', '79.24']
['h4KuX', None, '3 PCS', '70.95', '55.71', '2.59']
['2TEzi', 'Awesome Cheese', '26 PCS', '11.68', '58.4', '92.88']
['5bHoA', 'Awesome Fish', '33 PCS', '22.12', '75.97', '22.16']
['9VbSr', 'Rustic Metal Compute

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  80.0 %	% Accuracy:  0.0 %
Time Taken: 16.367902994155884 seconds
-----------------------------------------------------
['2', '90965335705029', 'Steel Soap', '57', '99.89', '27.19']
['3', '28720207714354', 'Fantastic Keyboard', '85', '42.18', '17.25']
['4', '15055233014492', 'Licensed Concrete Ch', '23', '54.75', '44.97']
['5', '89607427315216', 'New Mouse', '27', '21.53', '38.5']
[None, None, None, None, 'Amount', '112.50']
[None, None, None, 'GST @ 7 %', None, '7.88']
[None, 'One hundred twenty dollars and thirty-elght cents only', None, 'Net Amount', None, None]
[None, None, None, None, None, None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  98.33333333333333 %	% Accuracy:  20.0 %
Time Taken: 21.992366790771484 seconds
-----------------------------------------------------
['1', '83404470955316', 'Rustic Stee

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  60.0 %	% Accuracy:  0.0 %
Time Taken: 18.53096866607666 seconds
-----------------------------------------------------
[None, '71 PCS', '16.77', None, '44.44']
[None, None, 'Goods Totat', None, '140.28']
['Details for payment vin hank transer', None, 'Less Discount Gocs Amount', '%', '140.28']
['Bank Name United Overseas Bank LimitedA/C Name :Naspac Marketing Pe Ltd', None, '7%GST atalinelGST', None, None]
['heueccn', None, None, None, 'Enof Dixament']
['RECEIVEDYGOODSINGOQRDERANDCONDITION All cheques should be', None, None, 'ISSUED/APPROVED', None]
['crossed and made payable to 3 Yishun Ring Road #01-79 Singapore 760103', 'Kim Eng Mini Supormarket', None, None, None]
['Naspac Marketing Pte Ltd 758 1955Cashlery G753 6888Oice', '6778108', None, None, None]
[None, None, None, None, None, None, None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Le

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  90.0 %	% Accuracy:  50.0 %
Time Taken: 17.091423511505127 seconds
-----------------------------------------------------
['1', 'For repair Shirt', '42 PCS', '28.43', '2.67']
[None, 'For repair Soap', '61 PCS', '71.56', '93.59']
[None, None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  83.33333333333333 %	% Accuracy:  0 %
Time Taken: 13.149044513702393 seconds
-----------------------------------------------------
['1B-XI-927-PJP', 'Chair', '52 PCS', '55.16', '74.4']
[None]

-----------------------------------------------------
Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  76.66666666666667 %	% Accuracy:  0.0 %
Time Taken: 22.927066564559937 seconds
-----------------------------------------------------
[None, '84 PCS', '32.7', None, '97.77']
['Incredible Frozen Ke', '56 PCS', '92.83', 

Test  1  Parameters
Rotation Angle:  0 	Enhance Level:  0
Results
% Similarity:  89.28571428571429 %	% Accuracy:  5.0 %
Time Taken: 33.16290235519409 seconds
-----------------------------------------------------
['d4Cvr', 'Mouse Pizza', '9 PCS', '46.41', '12.63', '1.29']
['83Miv 5Wgpv', None, '55 PCS', '75.68', '98.24', '47.73']
['9XChO', 'Towels Frozen Table', '12 PCS 80 PCS', '21.24 22.7', '61.46 25.77', '45.59 27.76']
['G8XIz', 'Generic Fresh Towels Soap', '79 PCS', '93.65', '26.82', '23.7']
['55XSe', None, '69 PCS', '60.45', '40.74', '38.53']
['g1vBp', 'Frozen Chair', '99 PCS', '12.32', '28.14', '98.55']
['5aMrd', 'Steel Cheese', '27 PCS', '11.9', '1.74', '58.67']
['T0hHc 5Boea', 'Practical Wooden Pan', '17 PCS', '64.9', '22.67', '69.92']
['3T5jp', 'Metal Soap', '93 PCS', '29.71', '66.43', '84.13']
[None, 'Practical Hat', '29 PCS', '26.88', '28.94', '51.62']
['J6GQe', 'Ball', '23 PCS', '31.51', '50.3', '18.42']
['3SnWA', 'Sleek Pants', '81 PCS', '0.91', '14.76', '4.66']
['V5Pem', '